In [1]:
%pylab inline

from scipy.optimize import root_scalar
from scipy.integrate import trapezoid

Populating the interactive namespace from numpy and matplotlib


First, let's make a "qubit". This isn't a rigorous definition of a qubit, instead, we only define it with everything we need for this experiment. To this end, I have encoded the qubit's rotation around the y-axis as an intrinsic property of the qubit. This means that the qubit is being rotated around the $y$-axis on a time scale, which gives the qubit an internal "probability" of being measured in the $\lvert 1 \rangle$ state, depending on what time (after the rotation begins) you measure the qubit.

In [2]:
class Qubit:
    def __init__(self, seed, alpha, beta, phi0, c):
        self.alpha = alpha
        self.beta = beta
        self.phi0 = phi0
        self.c = c
        
        self.prob = lambda t: alpha / 2 * (1 - np.cos(c * t + phi0)) + beta
        
        self.rng = np.random.default_rng(seed)
        
    def measure(self, t, n=1):
        # The n parameter allows us to run multiple measurements for the same time step
        # with one function call. This speeds up the computation a bit.
        v = self.rng.random(size=n)
        p = self.prob(t)
        
        # "less than" ensures the probability actually works as intended
        return np.where(v < p, 1, 0)

We are going to assume these qubits are "ideal", that is $\alpha = 1$ and $\beta = \phi_0 = 0$. Additionally, we assume that we also know the approximate period of $P(t)$, and that is approximately 6. You could do the math on what this means, but in essence, we also assume that $c=1$. This can be verified by the fact that we hope to find pi by finding the two times that the $\lvert 1 \rangle$ probability $= 0.5$, which should occur at $t = \pi/2c$ and $t=3\pi/2c$.

In [3]:
q = {}

for i in range(5):
    seed = 5 * i + 7 * (i % 3)
    print(f"Adding qubit with seed {seed}")
    q[i] = Qubit(seed, 1, 0, 0, 1)

Adding qubit with seed 0
Adding qubit with seed 12
Adding qubit with seed 24
Adding qubit with seed 15
Adding qubit with seed 27


Now we run the measurement. The paper indicates that they made 8192 ($2^{13}$) measurements on each qubit at **each** time step, from 0 to 6.3 with a step of 0.1. Thankfully I included an $n$ parameter in my qubits, so we can do all 8192 measurements with a single call. So we loop over the time steps, and then over the qubits to do it on all qubits.

In [4]:
time_steps = np.arange(0, 6.3, 0.1)

f = {}
for i in range(len(q)):
    vals = []
    n_measure = 8192
    for t in time_steps:
        m = q[i].measure(t, n_measure)
        frac = np.sum(m) / len(m)
        vals.append(frac)

    f[i] = np.asarray(vals)

Now engage the algorithm. We estimate $\alpha$ and $\beta$ as $\hat{\alpha}$ and $\hat{\beta}$. Then renormalize the fractions to have a range of 0 and 1. I then define the linear interpolation of the function to find our estimated $t_1$ and $t_2$. (Steps 1-3 of the algorithm, page 3 in the paper).

In [5]:
f1 = {}
f1_interp = {}
alpha_hat = {}
beta_hat = {}
for i in range(len(q)):
    beta_hat[i] = np.min(f[i])
    alpha_hat[i] = np.max(f[i]) - beta_hat[i]

    f1[i] = (f[i] - beta_hat[i]) / alpha_hat[i]
    f1_interp[i] = lambda x: np.interp(x, time_steps, f1[i])

    print(alpha_hat[i], beta_hat[i])

0.99951171875 0.0
0.999267578125 0.0
0.99951171875 0.0
0.99951171875 0.0
0.9993896484375 0.0


Now we want to find where $\tilde{f_1} \approx 0.5$, close to $t_1 = 1.5$ and $t_2 = 4.5$. (Step 4 in the paper). here I define an $f_2$ interpolation that is simply $f_1 - 0.5$ for root finding purposes, since the root finder looks for 0's.

In [6]:
t1 = {}
t2 = {}

for i in range(len(q)):
    f2 = lambda x: np.interp(x, time_steps, f1[i]) - 0.5

    t1[i] = root_scalar(f2, x0=1.5, x1=2).root
    t2[i] = root_scalar(f2, x0=4.5, x1=5).root

    print(t1[i], t2[i])

1.572289156626506 4.690566037735849
1.5729216152019003 4.727067669172933
1.588858695652174 4.713594470046083
1.5712621359223304 4.702582159624414
1.5813758389261747 4.692077922077922


Next we refine our estimates of $\hat{\alpha}$ and $\hat{\beta}$ and renormalize the function, generating a new interpolated function. (Steps 5 and 6 in the paper). When finding these new estimates we require a "refinement" step parameter $\delta$ which in the paper is chosen to be 0.1.

In [7]:
delta = 0.1

for i in range(len(q)):
    t_maxval = (t1[i] + t2[i]) / 2
    t_minval = t_maxval + (t2[i] - t1[i])

    # Indices of time steps that satisfy this condition
    tmax = np.where(np.abs((time_steps - t_maxval)) < delta)[0]
    tmin = np.where(np.abs((time_steps - t_minval)) < delta)[0]

    beta_hat[i] = np.mean(f1[i][tmin])
    alpha_hat[i] = np.mean(f1[i][tmax]) - beta_hat[i]

    f1[i] = (f[i] - beta_hat[i]) / alpha_hat[i]
    f1_interp[i] = lambda x: np.interp(x, time_steps, f1[i])

    print(alpha_hat[i], beta_hat[i])

0.9979848558866634 0.0018319491939423546
nan nan
0.9982901807523205 0.00134342940889106
0.9985344406448461 0.0010991695163654128
0.9987174789300111 0.0010993037742762918


/Users/dylan/miniforge3/envs/tf/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dylan/miniforge3/envs/tf/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


We have to refine our estimates of $t_1$ and $t_2$. This step (Step 7 in the paper) is three sub steps: Find all $t$ within 0.5 of the $t_i$, fit a linear function to the $f_1$ values for those $t$ values, and find where that linear fit equals $0.5$.

In [8]:
t1_hat = {}
t2_hat = {}
for i in range(len(q)):
    # Indices of time steps that fulfill the above condition.
    t1_idx = np.where(np.abs((time_steps - t1[i])) < 0.5)[0]
    coeff_1 = np.polyfit(time_steps[t1_idx], f1[i][t1_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_1 = lambda x: coeff_1[0] * x + coeff_1[1] - 0.5
    t1_hat[i] = root_scalar(fit_1, x0=1.5, x1=2).root

    # Indices of time steps that fulfill the above condition.
    t2_idx = np.where(np.abs((time_steps - t2[i])) < 0.5)[0]
    coeff_2 = np.polyfit(time_steps[t2_idx], f1[i][t2_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_2 = lambda x: coeff_2[0] * x + coeff_2[1] - 0.5
    t2_hat[i] = root_scalar(fit_2, x0=1.5, x1=2).root


    print(t1_hat[i], t2_hat[i])

1.5666859991869053 4.702086556168658
nan nan
1.5779252914380246 4.712771133336845
1.570488050668516 4.7085897365398015
1.5764004718819755 4.705897634851899


Finaly we estimate the integral between $t_1$ and $t_2$ using the trapezoidal rule, which then gives us an estimate of $\pi$ via $\pi \approx \frac{t_2 - t_1}{I}$ (Steps 8 and 9 in the paper).

In [9]:
pi_guess = {}
for i in range(len(q)):
    f_hat = lambda x: np.interp(x, time_steps, f[i])

    # Adding the t1 and t2 to the measured timesteps for integration purposes.
    t_idx = np.where((time_steps >= t1[i]) & (time_steps <= t2[i]))[0]
    t_integ = time_steps[t_idx]
    t_integ = np.insert(t_integ, 0, t1_hat[i])
    t_integ = np.append(t_integ, t2_hat[i])

    # Doing the same for the probability measured values
    y_integ = f[i][t_idx]
    y_integ = np.insert(y_integ, 0, f_hat(t1_hat[i]))
    y_integ = np.append(y_integ, f_hat(t2_hat[i]))

    # Trapezoidal rule on the given points
    I = np.trapz(y_integ - 0.5, t_integ)

    # And the final guess!
    pi_guess[i] = (t2_hat[i] - t1_hat[i]) / I
    print(pi_guess[i])

3.1458002668465608
nan
3.144696696987822
3.1402728197073904
3.1483721728841703


Let's average up all our guesses now to improve our overall guess (we hope).

In [10]:
pi_init = np.nanmean(list(pi_guess.values()))
pi_init, 100 * (np.pi - np.nanmean(list(pi_guess.values()))) / np.pi

(3.1447854891064857, -0.10163111099219804)

That's not so bad, although granted this was an ideal case, and we used more qubits than in the paper (by 1). The paper uses 3 of the 5 experimental qubits, where here we ended up keeping 4 of the qubits, one of which we discarded because we could not estimate $t_{minval}$ to be within the experimental range. Which, ironically, makes this an even better simulation of the paper than expected, although granted we threw away the qubit for different reasons.

We might be tempted to see what would happen if we had, I don't know, 500 qubits. Let's see how this improves our guess.

In [11]:
q = {}
pi_guess = {}
time_steps = np.arange(0, 6.3, 0.1)

for i in range(500):
    seed = 5 * i + 7 * (i % 3)
    print(f"Adding qubit with seed {seed}")
    q[i] = Qubit(seed, 1, 0, 0, 1)

    vals = []
    n_measure = 8192
    for t in time_steps:
        m = q[i].measure(t, n_measure)
        frac = np.sum(m) / len(m)
        vals.append(frac)

    f = np.asarray(vals)  
    
    beta_hat = np.min(f)
    alpha_hat = np.max(f) - beta_hat

    f1 = (f - beta_hat) / alpha_hat
    f1_interp = lambda x: np.interp(x, time_steps, f1)
    
    f2 = lambda x: np.interp(x, time_steps, f1) - 0.5

    t1 = root_scalar(f2, x0=1.5, x1=2).root
    t2 = root_scalar(f2, x0=4.5, x1=5).root
    
    t_maxval = (t1 + t2) / 2
    t_minval = t_maxval + (t2 - t1)

    # Indices of time steps that satisfy this condition
    tmax = np.where(np.abs((time_steps - t_maxval)) < delta)[0]
    tmin = np.where(np.abs((time_steps - t_minval)) < delta)[0]

    beta_hat = np.mean(f1[tmin])
    alpha_hat = np.mean(f1[tmax]) - beta_hat

    f1 = (f - beta_hat) / alpha_hat
    f1_interp = lambda x: np.interp(x, time_steps, f1)
    
    # Indices of time steps that fulfill the above condition.
    t1_idx = np.where(np.abs((time_steps - t1)) < 0.5)[0]
    coeff_1 = np.polyfit(time_steps[t1_idx], f1[t1_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_1 = lambda x: coeff_1[0] * x + coeff_1[1] - 0.5
    t1_hat = root_scalar(fit_1, x0=1.5, x1=2).root

    # Indices of time steps that fulfill the above condition.
    t2_idx = np.where(np.abs((time_steps - t2)) < 0.5)[0]
    coeff_2 = np.polyfit(time_steps[t2_idx], f1[t2_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_2 = lambda x: coeff_2[0] * x + coeff_2[1] - 0.5
    t2_hat = root_scalar(fit_2, x0=1.5, x1=2).root
    
    f_hat = lambda x: np.interp(x, time_steps, f)

    # Adding the t1 and t2 to the measured timesteps for integration purposes.
    t_idx = np.where((time_steps >= t1) & (time_steps <= t2))[0]
    t_integ = time_steps[t_idx]
    t_integ = np.insert(t_integ, 0, t1_hat)
    t_integ = np.append(t_integ, t2_hat)

    # Doing the same for the probability measured values
    y_integ = f[t_idx]
    y_integ = np.insert(y_integ, 0, f_hat(t1_hat))
    y_integ = np.append(y_integ, f_hat(t2_hat))

    # Trapezoidal rule on the given points
    I = np.trapz(y_integ - 0.5, t_integ)

    # And the final guess!
    pi_guess[i] = (t2_hat - t1_hat) / I
    
np.nanmean(list(pi_guess.values()))

Adding qubit with seed 0
Adding qubit with seed 12
Adding qubit with seed 24
Adding qubit with seed 15
Adding qubit with seed 27
Adding qubit with seed 39
Adding qubit with seed 30
Adding qubit with seed 42
Adding qubit with seed 54
Adding qubit with seed 45
Adding qubit with seed 57
Adding qubit with seed 69
Adding qubit with seed 60
Adding qubit with seed 72
Adding qubit with seed 84
Adding qubit with seed 75
Adding qubit with seed 87
Adding qubit with seed 99
Adding qubit with seed 90
Adding qubit with seed 102
Adding qubit with seed 114
Adding qubit with seed 105
Adding qubit with seed 117
Adding qubit with seed 129
Adding qubit with seed 120
Adding qubit with seed 132
Adding qubit with seed 144
Adding qubit with seed 135
Adding qubit with seed 147
Adding qubit with seed 159
Adding qubit with seed 150
Adding qubit with seed 162
Adding qubit with seed 174
Adding qubit with seed 165
Adding qubit with seed 177
Adding qubit with seed 189
Adding qubit with seed 180
Adding qubit with see

3.14213981812504

Let's break down the reality a bit and see how it affects our results. For this first test, let's mess with $\alpha$ a little bit. Let's just adjust it a bit, with a little noise so that $0.8 \leq \alpha \leq 1.2$

In [12]:
q = {}
pi_guess = {}
time_steps = np.arange(0, 6.3, 0.1)
rng = np.random.default_rng(seed=91701)

for i in range(5):
    seed = 5 * i + 7 * (i % 3)
    alpha = 1 + (rng.random() * 0.4 - 0.2)
    print(f"Adding qubit with seed {seed}, alpha {alpha}")
    q[i] = Qubit(seed, 1, 0, 0, 1)

    vals = []
    n_measure = 8192
    for t in time_steps:
        m = q[i].measure(t, n_measure)
        frac = np.sum(m) / len(m)
        vals.append(frac)

    f = np.asarray(vals)  
    
    beta_hat = np.min(f)
    alpha_hat = np.max(f) - beta_hat

    f1 = (f - beta_hat) / alpha_hat
    f1_interp = lambda x: np.interp(x, time_steps, f1)
    
    f2 = lambda x: np.interp(x, time_steps, f1) - 0.5

    t1 = root_scalar(f2, x0=1.5, x1=2).root
    t2 = root_scalar(f2, x0=4.5, x1=5).root
    
    t_maxval = (t1 + t2) / 2
    t_minval = t_maxval + (t2 - t1)

    # Indices of time steps that satisfy this condition
    tmax = np.where(np.abs((time_steps - t_maxval)) < delta)[0]
    tmin = np.where(np.abs((time_steps - t_minval)) < delta)[0]

    beta_hat = np.mean(f1[tmin])
    alpha_hat = np.mean(f1[tmax]) - beta_hat

    f1 = (f - beta_hat) / alpha_hat
    f1_interp = lambda x: np.interp(x, time_steps, f1)
    
    # Indices of time steps that fulfill the above condition.
    t1_idx = np.where(np.abs((time_steps - t1)) < 0.5)[0]
    coeff_1 = np.polyfit(time_steps[t1_idx], f1[t1_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_1 = lambda x: coeff_1[0] * x + coeff_1[1] - 0.5
    t1_hat = root_scalar(fit_1, x0=1.5, x1=2).root

    # Indices of time steps that fulfill the above condition.
    t2_idx = np.where(np.abs((time_steps - t2)) < 0.5)[0]
    coeff_2 = np.polyfit(time_steps[t2_idx], f1[t2_idx], 1)

    # Once again subtracting 0.5 to find where the function = 0.5 not 0.
    fit_2 = lambda x: coeff_2[0] * x + coeff_2[1] - 0.5
    t2_hat = root_scalar(fit_2, x0=1.5, x1=2).root
    
    f_hat = lambda x: np.interp(x, time_steps, f)

    # Adding the t1 and t2 to the measured timesteps for integration purposes.
    t_idx = np.where((time_steps >= t1) & (time_steps <= t2))[0]
    t_integ = time_steps[t_idx]
    t_integ = np.insert(t_integ, 0, t1_hat)
    t_integ = np.append(t_integ, t2_hat)

    # Doing the same for the probability measured values
    y_integ = f[t_idx]
    y_integ = np.insert(y_integ, 0, f_hat(t1_hat))
    y_integ = np.append(y_integ, f_hat(t2_hat))

    # Trapezoidal rule on the given points
    I = np.trapz(y_integ - 0.5, t_integ)

    # And the final guess!
    pi_guess[i] = (t2_hat - t1_hat) / I
    
np.nanmean(list(pi_guess.values()))

Adding qubit with seed 0, alpha 0.861889890860902
Adding qubit with seed 12, alpha 0.8397242696678154
Adding qubit with seed 24, alpha 1.1199462015564432
Adding qubit with seed 15, alpha 1.0366580232944254
Adding qubit with seed 27, alpha 0.8921564065416642


/Users/dylan/miniforge3/envs/tf/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dylan/miniforge3/envs/tf/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


3.1447854891064857